# Working with Database Objects in IBM Db2 on Cloud

In the first two notebooks we explored Chicago data using local SQLite.  
In this part we move to the cloud DBMS **IBM Db2 on Cloud** to demonstrate how to create and use database objects:

- **VIEW** — views for convenient data access.  
- **STORED PROCEDURE** — stored procedures to encapsulate logic.  
- **TRANSACTION** — transaction examples using `COMMIT` and `ROLLBACK`.  

---

### Connecting to Db2  

Initially I tried to connect to the cloud directly from Jupyter Notebook using:

```bash
!pip install ibm_db_sa==0.3.3
!pip install sqlalchemy==1.3.24
!pip install ipython-sql
%load_ext sql
%sql ibm_db_sa://USER:PASSWORD@HOST:PORT/BLUDB?security=SSL

```

The attempt to connect to Db2 from Jupyter Notebook revealed compatibility issues of the `ibm_db` and `sqlalchemy` packages with Python 3.12.  
Therefore, creation of database objects (**VIEW, PROCEDURE, TRANSACTION**) was carried out via **Db2 Console → Run SQL**, and the results were saved as screenshots. 


## 1. Using JOINs

**Task:**  
Write an SQL query to display school names, community names, and average attendance for communities with a hardship index of 98.  
Also, create a query to display all crimes that occurred in schools (case number, crime type, community name).

**Code:** executed via `SELECT ... JOIN ...`


> The query was executed in the **external Db2 Console → Run SQL**.  
> The screenshot below shows the result of the execution.


![результат запроса](screens/join_result.png)


**Result:**  
For the **Riverdale** area (Hardship Index = 98) data were retrieved for four schools:  
- George Washington Carver Military Academy High School — attendance 91.6%  
- George Washington Carver Primary School — attendance 90.9%  
- Ira F Aldridge Elementary School — attendance 92.9%  
- William E B Dubois Elementary School — attendance 93.3%  

**Conclusion:**  
Using `JOIN` allowed combining school data with the district’s social indicators.  
It can be seen that even under extremely unfavorable conditions (high Hardship Index), Riverdale schools maintain a relatively high attendance level (90–93%).  
This shows that educational institutions remain resilient even in socially vulnerable areas.  
Thus, JOIN queries make it possible to comprehensively analyze the relationships between social and educational indicators, which is especially important for assessing vulnerable neighborhoods.


## 2. Crimes in Schools  

**Task:**  
Write an SQL query to display all crimes that occurred in schools. Include the case number, type of crime, and community name.  

**Code:** executed via `SELECT ... JOIN ...`


> The query was executed in the **external Db2 Console → Run SQL**.  
> The screenshot below shows the execution result.


![результат запроса](screens/join_result1.png)

**Results:**  
The selection shows incidents that occurred near or inside schools. Among them are:

- drug-related crimes (**NARCOTICS**),  
- violent acts (**BATTERY, ASSAULT**),  
- property damage (**CRIMINAL DAMAGE**),  
- trespassing incidents (**CRIMINAL TRESPASS**).  

**Conclusion:**  
Even school grounds are not completely safe: both minor offenses and serious crimes are recorded.  
These data highlight the need for additional security measures in educational institutions and in the surrounding areas.


## 3. Creating a View (VIEW)

**Task:**  
Create a `VIEW` with renamed columns.  
Verify its functionality by running:  
- a `SELECT * FROM <view>` query,  
- a query returning only school names and leader ratings.  

**Code:** executed via `CREATE OR REPLACE VIEW ...`


> The query was executed in **Db2 Console → Run SQL**.  
> Below is the result.

Db2 Console confirmed that the view was successfully created *(Status = ✅)*.

![результат запроса](screens/view.png)

SQL query to return all columns from the view:

**Result:**  
A table with renamed fields (School_Name, Safety_Rating, Family_Rating, etc.).

> **Column mapping (base table → view)**  

| Source (`CHICAGO_PUBLIC_SCHOOLS` table) | In the `CHICAGO_PUB_SCHOOLS` view |
|---|---|
| `NAME_OF_SCHOOL` | `School_Name` |
| `Leaders_Icon`   | `Leaders_Rating` |
| `Safety_Icon`    | `Safety_Rating` |
| `Family_Involvement_Icon` | `Family_Rating` |
| `Environment_Icon` | `Environment_Rating` |
| `Instruction_Icon` | `Instruction_Rating` |
| `Teachers_Icon` | `Teachers_Rating` |

> ℹ️ **Note:** in the original table the field **`Leaders_Icon`** is updated,  
> while in the view it appears under the alias **`Leaders_Rating`**.


![результат запроса](screens/view1.png)

SQL query that returns only the **school name** and the **leaders rating** from the view.


Result: a list of schools with their leaders’ ratings.


![результат запроса](screens/view2.png)

***Conclusion***

Using a VIEW allows you to:
- simplify the table structure by assigning clear column names;
- hide technical details of the database;
- create selections with user-friendly fields for analysts and end-users.

Thus, views make working with data easier and queries more readable.


## 4. Stored Procedure `UPDATE_LEADERS_SCORE`

**Task:**  
Create or replace the procedure `UPDATE_LEADERS_SCORE` with the following parameters:  
- `in_School_ID` (INTEGER)  
- `in_Leader_Score` (INTEGER)  

Inside the procedure, an `IF` statement is used to update the `Leaders_Icon` field in the `CHICAGO_PUBLIC_SCHOOLS` table.

**Details:**  
- Before creating the procedure, the `SET TERMINATOR @` command is used.

**Test:**  
Calling `CALL UPDATE_LEADERS_SCORE(<School_ID>, 50)` should change the value to **AVERAGE**.



> The query was executed in the **external Db2 Console** → *Run SQL*.

Db2 Console confirmed the successful creation of the procedure *(Status = ✅)*.


![результат запроса](screens/storage_procedure.png)

***Db2 Console Interface***

This is how the created procedure appears in the list of **Stored Procedures** along with its parameters.


![результат запроса](screens/storage_proc1.png)

***Adding Conditions with IF***

To update the text field **Leaders_Icon** depending on the range of **in_Leader_Score**, the following code is used:


After writing the code shown above, we executed it in **Db2 Console**.  
The screenshot below shows the successful creation of the `UPDATE_LEADERS_SCORE` procedure with all added conditions and transaction support.


![результат запроса](screens/storage_proc2.png)

***Procedure Check***

Write a query to call the stored procedure, passing a valid school ID and a leader score of **50** to verify that the procedure works as expected.


![результат запроса](screens/storage_proc3.png)

**Verification Result**


After calling the procedure, we run a verification query to ensure that the changes were actually applied.  
In the screenshot below, you can see that for **Abraham Lincoln Elementary School** the value of the `LEADERS_ICON` field has changed to *Average*.


![результат запроса](screens/storage_proc4.png)

In the screenshot above, you can see that the `LEADERS_ICON` field for **Abraham Lincoln Elementary School** has changed to *Average*.

**Conclusion:**  
For **Abraham Lincoln Elementary School**, the `Leaders_Icon` field value changed to *AVERAGE* (previously *STRONG*).  
This confirms that the procedure correctly updates leadership indicators based on the input score.  
We now have a working procedure that can be applied to any school, making updates easier to automate and making data handling more reliable and scalable.


### 5. Using Transactions

**Task:**  
Refine the `UPDATE_LEADERS_SCORE` procedure by adding:  
- a universal `ELSE` block that rolls back changes using `ROLLBACK`,  
- a final transaction confirmation using `COMMIT`.

**Tests:**  
1. Call with a valid value (e.g., `38`) → the update succeeds.  
2. Call with an invalid value (e.g., `101`) → `ROLLBACK` is triggered, no changes are applied.


**Input validation rules**
- Allowed `in_Leader_Score` values: **0–99**.
- If the value is out of range or `NULL`, the safeguard triggers: **`ROLLBACK`** (no changes applied).
- A successful update is finalized with an explicit **`COMMIT`**.

This guarantees the procedure won’t write invalid data and preserves table integrity.


Below is the full code of the `UPDATE_LEADERS_SCORE` stored procedure, including an `ELSE` block to roll back the transaction (`ROLLBACK`) and a final commit (`COMMIT`).

After running the statement, Db2 Console confirms the procedure was created successfully:


![результат запроса](screens/update_procedure.png)

**Test 1.**  
Run a query to verify that the updated stored procedure works as expected when we pass a valid score of **38**.


![результат запроса](screens/update_proc1.png)

The procedure executed successfully *(Status = ✅)*.  
Now let's check the `CHICAGO_PUBLIC_SCHOOLS` table to ensure that the `Leaders_Icon` field has been updated for the selected school.


![результат запроса](screens/check1.png)

For **Abraham Lincoln Elementary School**, the value of the `LEADERS_ICON` field changed to **Weak** (previously **Average**).  
This confirms that the procedure works correctly with a valid input value.


**Test 2.**  
Run another query to verify that the updated stored procedure behaves as expected when we use an **invalid score (101)**.


![результат запроса](screens/update_proc2.png)

The procedure executed successfully *(Status = ✅)*.  
However, because the invalid score `101` was passed, the `ELSE` block with the transaction rollback (`ROLLBACK`) should have been triggered.  

Now run a verification query to ensure that the data in the table has **not** changed.


![результат запроса](screens/check2.png)

The value of the `LEADERS_ICON` field at **Abraham Lincoln Elementary School** did **not** change (remained *Weak*),  
because when an invalid score was passed, the `ELSE` block was triggered and the transaction was rolled back (**ROLLBACK**).


## Summary

In Part III of the project, the key database operations were implemented and successfully tested:

- **JOIN queries** were written to combine data from multiple tables.  
- A **VIEW** was created to simplify access to frequently used fields.  
- A **stored procedure** was developed and tested to update leadership scores.  
- **Transaction control** with `ROLLBACK` and `COMMIT` was added to ensure data integrity.  

All objects function correctly and are visible under the **Objects** tab in the Db2 Console.  
This part of the project demonstrates not only SQL query skills but also the ability to design procedures with transaction control — an important step toward building reliable and scalable database solutions.



## How to Reproduce

1. Open **Db2 Console → Run SQL** in your own Db2 on Cloud instance.
2. Execute the DDL statements in the following order:
   1) `CREATE VIEW CHICAGO_PUB_SCHOOLS ...`
   2) `CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE ...` (with `--#SET TERMINATOR @`)
3. Test the procedure:
   - **Positive scenario:**  
     ```sql
     CALL UPDATE_LEADERS_SCORE(610038, 38);
     SELECT NAME_OF_SCHOOL, Leaders_Icon
     FROM CHICAGO_PUBLIC_SCHOOLS
     WHERE School_ID = 610038;
     ```
   - **Negative scenario (validation):**  
     ```sql
     CALL UPDATE_LEADERS_SCORE(610038, 101);
     SELECT NAME_OF_SCHOOL, Leaders_Icon
     FROM CHICAGO_PUBLIC_SCHOOLS
     WHERE School_ID = 610038;
     ```
4. If needed, refresh the view or run `SELECT` on `CHICAGO_PUB_SCHOOLS` to see the aliased columns (`Leaders_Rating`, etc.).

